<font size = 6><h1 align = center>Monte Carlo Fun </h1 ></font>      

<font size = 4><h2 align = center> Derek Sikorski </font><h2>

---
---
---


<font size = 5><h1 align = center>File Summary</h1 ></font>      

**Purpose:** 
This file is meant to handle MCing data used for the Hyperion SMF project. The data broadly includes:
1. COSMOS2020 photometry
2. Assorted ground-based spectroscopy
3. HST grism data

**Outputs:**
This code produces two separate catalogs of MCed redshifts. One is for the COSMOS photometry while the other is for any other is for any other assortment of observations. The logic is that the photoz's are drawn completely randomly for all galaxies in the sample whereas the galaxies with other observations maybe be chosen by a variety of functions. Therefore, it is easiest to simply replace the redshifts of the galaxies with extra observations in place.

**General Logic**


---
---

In [2]:
import numpy as np
from scipy.stats import skewnorm
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from astropy.io import fits
import os
from tqdm.notebook import tqdm

<font size = 5><h1 align = center>MC Functions</h1 ></font>      

Create the MC function and it's plotting functions

In [2]:
def MCz(niter, zs, weights, z_range, MC_fn, plot_field="", plot_zrange="", verbose=False, **kwargs):
    """
    Performs a Monte Carlo on the redshift distribution of input galaxies

    INPUTS:
        - niter (int)   = Number of MC iterations to run
        - zs (array)    = List of median redshift values
        - weights (array)   = List of the MC weights for each object. A new z is drawn from the PDF if random_number >= weight
        - z_range (array)   = Range of redshifts to keep
        - MC_fun (fn)   = Python function used to generate the new redshift values for the galaxies
        - plot_field (str)    = Path to the directory where plots should be saved. If left as "", then no plots are saved
        - plot_zrange (str)    = Path to the directory where plots of galaxies in z_range should be saved.
        - verbose (bool)    = If you want to print the status bar via tqdm.notebook
        - **kwargs = For the MC_fun
        
    OUTPUTS:
        - (array) --> Indices in redshift array of objects falling in z_range at least once
        - (array) --> 2D array of redshifts of shape (len(zs), niter)
        - (array) --> 2D array of of indices which failed the MC draw (i.e. new z was drawn)
    """
    ## Setup for MC
    new_zs = [] # Fill with new redshifts
    iterable = tqdm(range(niter)) if verbose else range(niter)      # What to iterate over in for-loop based on 'verbose' option
    drawn_idxs = [] # Fill with idxs that were drawn

    ## Run the MC iterations
    for n in iterable:
        z_in = np.copy(zs)      # Copy of redshifts to manipulate

        ## Pick galaxies to get new z's and change where need
        new_idxs = np.where( np.random.random(size=len(zs)) >= weights )    # Which z's to change
        nz = MC_fn(*kwargs.values())            # Pick new z's
        z_in[new_idxs] = nz[new_idxs]    # Replace redshifts where needed

        new_zs.append(z_in)      # Add to list of redshifts   
        drawn_idxs.append(new_idxs)


        ### PLOTTING ###
        if plot_field != "": MC_plot1(z_in, plot_field, n)
        if plot_zrange != "": MC_plot2(z_in, z_range, plot_zrange, n)

    new_zs = np.array(new_zs)   # MCed redshifts

    ## Find which galaxies fell in z-range at least once ###
    z_bool = ((z_range[0]< new_zs) & (new_zs < z_range[1])).any(axis=0)
    good_idxs = np.where(z_bool)[0]     # Where the condition is met

    return good_idxs, new_zs.transpose(), drawn_idxs

In [3]:
def MC_plot1(z_in, plot_field, n):
    """
    Generate plot of the MC distribution over all redshifts from 0-8
    """
    fig, ax = plt.subplots()
    bbox = dict(boxstyle='round', fc = "white", ec='k', alpha=0.5)
    ax.hist(z_in, bins=np.arange(0,8,0.05))
    ax.set_title(f"Redshift Distribution of C20 -- (MC_iter {n})")
    ax.text(0.7,0.9, f"# of Galaxies = {len(np.where((0 <=z_in) & (10>= z_in))[0])}", fontsize=7, bbox=bbox, transform=ax.transAxes)
    ax.set_xlabel("z")
    ax.set_ylabel("N")
    try:
        fig.savefig(plot_field + f"run_{n}")
    except:
        os.mkdir(plot_field)
        fig.savefig(plot_field + f"run_{n}")
    plt.close()

In [4]:
def MC_plot2(z_in, z_range, plot_zrange, n):
    """
    Generate a plot of the MC distribution given a specific range of redshifts
    """
    fig, ax = plt.subplots()
    ax.hist(z_in, bins=np.arange(2,3,0.01))

    ax.set_title(f"Redshift Distribution of Field -- (MC_iter {n})")
    bbox = dict(boxstyle='round', fc = "white", ec='k', alpha=0.5)  # Textbox
    ax.text(0.7, 0.9, f"# of Galaxies = {len(np.where((z_range[0] <=z_in) & (z_range[1]>= z_in))[0])}", 
            fontsize=7, bbox=bbox, transform=ax.transAxes)
    ax.set_xlabel("z")
    ax.set_ylabel("N")
    try:
        fig.savefig(plot_zrange + f"run_{n}")
    except:
        os.mkdir(plot_zrange)
        fig.savefig(plot_zrange + f"run_{n}")
    plt.close()

Create the asymmetric normal distribution to draw the photoz's from

In [5]:
def my_PDF(alpha, omega, loc):
    """
    PDFs to draw the new redshifts from. Skewed-normal based on the confidence interval from COSMOS2020

    INPUTS:
        - xs (array)    = Median redshift values
        - l68 (array)   = Lower bound of the 68% confidence interval
        - u68 (array)   = Upper bound of the 68% confidence interval
    OUTPUTS:
        - (array)   = New redshift values. redshifts <0 or unavailable are marked -99
    """
    z_vals = skewnorm.rvs(a=alpha, loc=loc, scale=omega) # Find new zs based on skew-normal
    return z_vals

----
---
---

# Fitting Skew-Normals

In [6]:
def costFn(params, z_params):
    alpha, omega, loc = params
        
    dist = skewnorm(alpha, scale=omega, loc=loc)

    # Calculate the CDF at points a, b, and c
    cdf_a = dist.cdf(z_params[1])
    cdf_b = dist.cdf(z_params[0])
    cdf_c = dist.cdf(z_params[2])
    
    # The areas we want to match
    area_left = cdf_b - cdf_a
    area_right = cdf_c - cdf_b
    
    # Objective is to make both areas equal to 0.34
    return (area_left - 0.34)**2 + (area_right - 0.34)**2 + (dist.median()-z_params[0])**2

In [7]:
def fit_Plot(af, wf, lf, zmed, l68, u68, plot_path, cid):

    dist = skewnorm(af, scale=wf, loc=lf)
    rvs = dist.rvs(10000)

    delta =af / np.sqrt(1 +af**2)
    gamma = (4-np.pi)/2 * (delta*np.sqrt(np.pi/2))**3 / (1-2*delta**2/np.pi)**1.5
    mu = dist.mean()
    final_med = dist.median()

    # Final skew-normal distribution
    p_l68 = dist.cdf(zmed) - dist.cdf(l68)
    p_u68 = dist.cdf(u68) - dist.cdf(zmed)
    xs = np.linspace(zmed-3*(zmed-l68), zmed+3*(u68-zmed), 1000)
    plt.figure(figsize=(7, 5))
    plt.title(rf"C20 ID = {int(cid)}    $\alpha$={round(af, 2)}       $\omega$={round(wf, 2)}      $\xi$={round(lf, 2)}    $\gamma$={round(gamma, 2)}")
    plt.hist(rvs, density=True, bins=100, label=f"N = {10000} draws", alpha=0.75)
    plt.plot(xs, dist.pdf(xs))
    plt.vlines(mu, ymin=0, ymax=2, color='red', label=f"Mean = {round(mu, 3)}")
    plt.vlines(final_med, ymin=0, ymax=2, color='orange', label=f"Median = {round(final_med, 3)}    ({zmed})")
    plt.vlines(l68, ymin=0, ymax=1, color='g', label=f"Lower-bound  ({round(p_l68,3)})")
    plt.vlines(u68, ymin=0, ymax=1, color='m', label=f"Upper-bound  ({round(p_u68, 3)})")
    plt.legend()
    plt.savefig(plot_path)
    plt.close()

In [8]:
def fitDist(zmed, l68, u68, plot_path = "", cid = None):
    
    wi = 2/3*(u68 - l68)    # Initial omega
    var = ((zmed-l68)**2 + (u68-zmed)**2)/2     # variance estimate
    d = np.sqrt(  np.pi/2 * (1-var/wi**2))      # Delta parameter

    if (zmed - l68) < (u68-zmed):
        li = l68 -wi/4
        ai = 50*var**0.5
    else:
        li = u68 + wi/4
        ai = -50*var**0.5

    # Initial guess for omega and alpha
    initial_guess = [ai, wi, li]

    # Minimize the objective function
    result = minimize(costFn, initial_guess, tol=1e-14, options={'maxiter':100},
                    args=([zmed, l68, u68]), bounds = ((-12, 12), (0,5), (0,7)))
    
    af, wf, lf = result.x   # Optimized parameters
    residual = costFn(result.x, [zmed, l68, u68])

    if plot_path != "": fit_Plot(af, wf, lf, zmed, l68, u68, plot_path, cid)

    return af, wf, lf, residual

In [9]:
## READ IN FILE ##
cosmos_file = fits.open(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/COSMOS2020_CLASSIC_R1_v2.0.fits")
c20p = cosmos_file[1].data

## FIND BAD GALAXIES ##
bad_ids = np.where((np.isnan(c20p["lp_zPDF"]) == True) |        # No redshift from lephare
                   (np.isnan(c20p["lp_zPDF_l68"]) == True) |    # No lower-68-percentile from lephare
                   (np.isnan(c20p["lp_zPDF_u68"]) == True))[0]  # no upper-68-percentile from lephare

print(f"Number of galaxies = {len(c20p)}")
print(f"Number of bad galaxies = {len(bad_ids)}")

## INSERT TEMP DATA IN BAD IDs ##
# c20p["lp_zPDF"][bad_ids] = 2
# c20p["lp_zPDF_l68"][bad_ids] = 1.9
# c20p["lp_zPDF_u68"][bad_ids] = 2.1

c20p_cut = np.delete(c20p, bad_ids)

Number of galaxies = 1720700
Number of bad galaxies = 19258


In [10]:
## CUT DATA TO POTENTIALLY USABLE ##
ra_range = (149.6, 150.52)  
dec_range = (1.74, 2.73)
IRAC_cut = 25.


g_idxs = np.where((c20p_cut["ALPHA_J2000"] >= ra_range[0]) & (c20p_cut["ALPHA_J2000"] <= ra_range[1])       # RA
                & (c20p_cut["DELTA_J2000"] >= dec_range[0]) & (c20p_cut["DELTA_J2000"] <= dec_range[1])     # DEC
                & ((c20p_cut["IRAC_CH1_MAG"] <= IRAC_cut) | (c20p_cut["IRAC_CH2_MAG"] <= IRAC_cut)) # IRAC
                & ((c20p_cut["lp_type"] == 0) | (c20p_cut["lp_type"] == 2)))        # LePhare type

g_c20p = c20p_cut[g_idxs]

In [11]:
g_c20p.shape

(287363,)

In [12]:
n = 3

med = g_c20p["lp_zPDF"]

diff_l = med - g_c20p["lp_zPDF_l68"]
diff_u = g_c20p["lp_zPDF_u68"] - med

l_crit = med - n*diff_l
u_crit = med + n*diff_u

in_2_3 = np.where(
    (  (med <= 3) & (u_crit >= 2) )
    | (  (med >= 2) & (l_crit <= 3) )
)

small_c20p = g_c20p[in_2_3]

In [18]:
print(small_c20p.shape[0]*0.25 /60)

382.9583333333333


In [14]:
### Just Plotting ###
for idx in tqdm(range(100)):
    c_index = small_c20p["ID"][idx]

    zm, l68, u68 = small_c20p["lp_zPDF"][idx], small_c20p["lp_zPDF_l68"][idx], small_c20p["lp_zPDF_u68"][idx]

    plot_path = f"zDists/c20_{c_index}.png"

    a, w, l, r = fitDist(zm, l68, u68, plot_path, c_index )


  0%|          | 0/100 [00:00<?, ?it/s]

c:\Users\sikor\Anaconda3\envs\py38\lib\site-packages\scipy\stats\_distn_infrastructure.py:2176: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
c:\Users\sikor\Anaconda3\envs\py38\lib\site-packages\scipy\stats\_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
c:\Users\sikor\Anaconda3\envs\py38\lib\site-packages\scipy\stats\_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\Users\sikor\AppData\Local\Temp\ipykernel_19324\3971317608.py:5: RuntimeWarning: invalid value encountered in sqrt
  d = np.sqrt(  np.pi/2 * (1-var/wi**2))      # Delta parameter


In [17]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
save_fits = np.zeros((small_c20p.shape[0], 5))

# for idx in tqdm(range(small_c20p.shape[0])):
for idx in tqdm(range(100)):

    c_index = small_c20p["ID"][idx]

    zm, l68, u68 = small_c20p["lp_zPDF"][idx], small_c20p["lp_zPDF_l68"][idx], small_c20p["lp_zPDF_u68"][idx]

    a, w, l, r = fitDist(zm, l68, u68)

    save_fits[idx] = np.array([c_index, a, w, l, r])

# np.save(f"zFits/zFits.npy", save_fits)

  0%|          | 0/100 [00:00<?, ?it/s]

---
---
---

# MC C20 Photometry

In [11]:
### Find good Fits ###
all_fits = np.load("zFits/zFits.npy")

# Find good fits based on proximity of median
med_diffs = []
for f in tqdm(all_fits):
    skn = skewnorm(f[1], scale=f[2], loc=f[3])
    z_med = c20p["lp_zPDF"][int(f[0])-1]
    med_diffs.append(skn.median() - z_med) 

b_fits = np.where(np.abs(med_diffs) > 0.1)    # bad residual

g_fits = np.delete(all_fits, b_fits, axis=0)

  0%|          | 0/70283 [00:00<?, ?it/s]

In [12]:
print("All fits: ", all_fits.shape[0])
print("Bad fits: ", b_fits[0].shape[0])
print("Good fits: ", g_fits.shape[0])


All fits:  70283
Bad fits:  234
Good fits:  70049


In [13]:
#### RUN THE MC ####
# ========================================================
# ========================================================
for run in range(1):
    niter = 100      # Number of iterations

    z_range = [2,3]         # Redshift range for 
    # plot_field = f"./MC_iterations/c20p_total_{run}/"
    # plot_zrange = f"./MC_iterations/c20p_Hyper_{run}/"

    phot_med = c20p["lp_zPDF"][g_fits[:,0].astype(int)-1]
    # phot_l68 = phot_med - c20p["lp_zPDF_l68"]
    # phot_u68 = c20p["lp_zPDF_u68"] - phot_med

    # ========================================================
    # ========================================================
    # ========================================================

    ## MC ##
    phot_ids, new_pzs, _ = MCz(niter ,phot_med, np.zeros(len(g_fits)), z_range, my_PDF,
                         verbose=True, alpha=g_fits[:,1], omega=g_fits[:,2], loc=g_fits[:,3])

    ## Update bad galaxies ##
    # new_pzs[bad_ids] = np.full(shape=(len(bad_ids), niter), fill_value=-99)

    ## WRITE TO RESULT FILE ##

    # Update dtypes
    dtypes = [c20p.dtype.descr[0]] + [(f"MC_iter{n}", ">f8") for n in range(niter)]

    # Make array to fill
    write_arr = np.zeros(shape=(len(g_fits)), dtype=dtypes)

    write_arr["ID"] = c20p["ID"][g_fits[:,0].astype(int)-1]
    for n in range(niter):
        write_arr[f"MC_iter{n}"] = new_pzs[:,n]

    np.save(rf"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/C20_MC_100_{run}.npy", write_arr)

  0%|          | 0/100 [00:00<?, ?it/s]

---

# MC Spectra and Grism 

In [14]:
## LOAD DATA ##
# PHOTO-Zs
cosmos_file = fits.open(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/COSMOS2020_CLASSIC_R1_v2.0.fits")
c20p = cosmos_file[1].data

# SPECTRA
specz_cat = np.loadtxt("./Data/master_specz_COSMOS_BF_v4b.cat", dtype=object)   # Load in the data
# Fix up the formatting for the spec data-file:
new_array = []
for idx in range(specz_cat.shape[1]):
    try:
        col = specz_cat[:,idx].astype(np.float32)
    except:
        col = specz_cat[:,idx]
    new_array.append(col)

c20s = np.array(new_array, dtype=object)
c20s = np.transpose(c20s)

miss_spec = np.where(c20s[:,0] == -99)[0]   # spectra not in the cosmos catalog

print(f"Number of C20 spectra: {c20s.shape[0]}")
print(f"Number of missing spectra: {len(miss_spec)}")

# ----------------------------------------------------------------------
# ----------------------------------------------------------------------

# GRISM
griz_cat = np.loadtxt("./Data/HST_Hyp_zcat.v1.2.cat", skiprows=1, usecols=range(16), dtype=object)   # Load in the data
# Fix up the formatting for the spec data-file:
new_array = []
for idx in range(griz_cat.shape[1]):
    try:
        col = griz_cat[:,idx].astype(np.float32)
    except:
        col = griz_cat[:,idx]
    new_array.append(col)

griz = np.array(new_array, dtype=object)
griz = np.transpose(griz)

miss_griz = np.where(griz[:,4] == -99)[0]   # spectra not in the cosmos catalog


print(f"Number of Grism redshifts: {griz.shape[0]}")
print(f"Number of missing grisms: {len(miss_griz)}")

Number of C20 spectra: 42776
Number of missing spectra: 2562
Number of Grism redshifts: 12764
Number of missing grisms: 53


In [37]:
## FIND COMMON OBJECTS ##
gids = []       # idx in grism catalog of common object
sids = []       # idx in spectrum catalog of common object 

sim_objs = []   # Keep track of info of the object for MC use --> [C20_ID, zs, qf_s, zg, qf_gz, bf]

for g_id, c_id in enumerate(griz[:,4]):
    if c_id > 0:    # Make sure it's a cosmos object
        t = np.where(c_id == c20s[:,0])[0]

        if len(t) != 0: # Object is in both
            gids.append(g_id)   # Add grism id
            sids.append(t[0])   # Add spec id
            sim_objs.append([c_id, griz[g_id][10], griz[g_id][11], griz[g_id][13], griz[g_id][15], griz[g_id][5]])


# Create unique catalogs
spec_unique = np.delete(c20s, sids, axis=0)
griz_unique = np.delete(griz, gids, axis=0)
sim_objs = np.array(sim_objs)

print(f"Unique Grizli Objects = {len(griz_unique)}")
print(f"Unique Spec Objects = {len(spec_unique)}")
print(f"Common Objects = {len(sim_objs)}")

Unique Grizli Objects = 10053
Unique Spec Objects = 40106
Common Objects = 2711


In [16]:
## PREP STORAGE ARRAY ## 
niter = 100

dtypes = [c20p.dtype.descr[0]] + [(f"MC_iter{n}", ">f8") for n in range(niter)]

# Make array to fill
spec_mc = np.zeros(shape=(len(spec_unique) + len(griz_unique) + len(sim_objs)), dtype=dtypes)

which_z = np.zeros(shape=(len(spec_unique) + len(griz_unique) + len(sim_objs)), dtype=dtypes)

## MC C20 Spectra

In [17]:
## Pack parameters of spectra from cosmos catalog ##
bad_spec = [] # where parameters have a nan
gal_params = []
for s_id, c_id in enumerate(spec_unique[:,0].astype(int)):

    if c_id == -99:     # Not a cosmos object
        bad_spec.append(s_id)
        gal_params.append([2, 1.9, 2.1])    # make up temporary parameters

    else:   # It is a cosmos object
        med = c20p["lp_zPDF"][c_id-1]
        l68 = med - c20p["lp_zPDF_l68"][c_id-1]
        u68 = c20p["lp_zPDF_u68"][c_id-1] - med


        if (med != med) or (l68 != l68) or (u68 != u68):   # p(z) contains a NaN
            bad_spec.append(s_id)
            gal_params.append([2, 1.9, 2.1])    # make up temporary parameters

        else:
            gal_params.append([med, l68, u68])

gal_params = np.array(gal_params)
bad_spec = np.array(bad_spec)
print(f"Number of bad p(z)'s for specta: {len(bad_spec)}")

Number of bad p(z)'s for specta: 3911


In [18]:
### Create small catalog of spec/ grism fits ###
s_fits = []

for s_id, s in tqdm(enumerate(c20s), total=len(c20s)):
    c_id = int(s[0])
    if c_id == -99: continue    # Skip if not cosmos number
    
    # Check if already been fit
    f_ids = np.where(g_fits[:,0].astype(int) == c_id)[0]
    if len(f_ids) != 0:    # Already been fit
        s_fits.append(g_fits[f_ids[0]])
        continue
    
    # Hasn't been fit
    else:
        a, w, l, r = fitDist(c20p["lp_zPDF"][c_id-1], c20p["lp_zPDF_l68"][c_id-1], c20p["lp_zPDF_u68"][c_id-1])
        #Check to see if fit converged
        f_med = skewnorm(a,scale=w,loc=l).median()
        if np.abs(f_med - c20p["lp_zPDF"][c_id-1] ) < 0.1:
            s_fits.append([c_id, a,w,l,r])
        else:
            continue

s_fits = np.array(s_fits)
np.save("zFits/sFits.npy", s_fits)

  0%|          | 0/42776 [00:00<?, ?it/s]

c:\Users\sikor\Anaconda3\envs\py38\lib\site-packages\scipy\stats\_distn_infrastructure.py:2176: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
c:\Users\sikor\Anaconda3\envs\py38\lib\site-packages\scipy\stats\_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
c:\Users\sikor\Anaconda3\envs\py38\lib\site-packages\scipy\stats\_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\Users\sikor\AppData\Local\Temp\ipykernel_22100\3998956261.py:5: RuntimeWarning: invalid value encountered in sqrt
  d = np.sqrt(  np.pi/2 * (1-var/wi**2))      # Delta parameter


TypeError: _save_dispatcher() missing 1 required positional argument: 'arr'

In [49]:
### Pack the fits for use ###
s_params = []
b_params = []
for idx, c_id in enumerate(spec_unique[:,0]):
    idxs = np.where(s_fits[:,0] == c_id)[0]
    if len(idxs) != 0:
        i = idxs[0]
        s_params.append([s_fits[i][1],s_fits[i][2], s_fits[i][3]])
    else :
        b_params.append(idx)
    

s_params = np.array(s_params)
final_spec = np.delete(spec_unique, b_params, axis=0)

In [52]:
#### RUN THE MC ####
# ========================================================
# ========================================================
z_range = [2,3]         # Redshift range for 
# plot_field = "./MC_iterations/c20s_total/"
# plot_zrange = "./MC_iterations/c20s_Hyper/"

spec_z = final_spec[:,11]         # orginal spec-z
spec_med = gal_params[:,0]  # parameters for the p(z)
spec_l68 = gal_params[:,1]
spec_u68 = gal_params[:,2]

# Set the MC weights based on the quality flags
qfs = final_spec[:,13] % 10      # 
spec_weights = np.select( [(qfs >=2.)&(qfs<3.),(qfs>=9.)&(qfs<10.), (qfs>=3.)&(qfs<5.) ],
                [0.7, 0.7, 0.993],
                default=0)
# ========================================================
# ========================================================
# ========================================================

## MC ##
spec_ids, new_szs, drawn_idxs = MCz(niter, spec_z, spec_weights, z_range, my_PDF,
                     verbose=True, alpha = s_params[:,0], omega = s_params[:,1], loc = s_params[:,2])

## Update bad galaxies ##
# new_szs[bad_spec] = np.full(shape=(len(bad_spec), niter), fill_value=-99)

## WRITE TO RESULT FILE ##

# Update dtypes
dtypes = [c20p.dtype.descr[0]] + [(f"MC_iter{n}", ">f8") for n in range(niter)]

# Make array to fill
write_arr = np.zeros(shape=(len(final_spec)), dtype=dtypes)

write_arr["ID"] = final_spec[:,0]
for n in range(niter):
    write_arr[f"MC_iter{n}"] = new_szs[:,n]
    new_ids = drawn_idxs[n]

    which_z[f"MC_iter{n}"][:len(new_szs)][new_ids] = np.ones(len(new_ids))

np.save(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/C20_spec_MC_1000.npy", write_arr)

spec_mc["ID"][:len(new_szs)] = final_spec[:,0]     # update with cosmos IDs
which_z[f"ID"][:len(new_szs)] = final_spec[:,0]


for n in range(niter):
    spec_mc[f"MC_iter{n}"][:len(new_szs)] = new_szs[:,n]

  0%|          | 0/100 [00:00<?, ?it/s]

---

## MC Grizli Spectra

In [53]:
## Pack parameters of spectra from cosmos catalog ##
bad_griz = [] # where parameters have a nan
griz_params = []
for g_id, c_id in enumerate(griz_unique[:,4].astype(int)):

    if c_id == -99:     # Not a cosmos object
        bad_griz.append(g_id)
        griz_params.append([2, 1.9, 2.1])    # make up temporary parameters

    elif griz_unique[:,5][g_id] == 1:   # blended object --> discard
        bad_griz.append(g_id)
        griz_params.append([2, 1.9, 2.1])

    else:   # It is a cosmos object
        med = c20p["lp_zPDF"][c_id-1]
        l68 = med - c20p["lp_zPDF_l68"][c_id-1]
        u68 = c20p["lp_zPDF_u68"][c_id-1] - med


        if (med != med) or (l68 != l68) or (u68 != u68):   # p(z) contains a NaN
            bad_griz.append(g_id)
            griz_params.append([2, 1.9, 2.1])    # make up temporary parameters

        else:
            griz_params.append([med, l68, u68])

griz_params = np.array(griz_params)
bad_griz = np.array(bad_griz)
print(f"Number of bad p(z)'s for specta: {len(bad_griz)}")

Number of bad p(z)'s for specta: 637


In [54]:
### Create small catalog of spec/ grism fits ###
griz_fits = []

for g_id, g in tqdm(enumerate(griz_unique), total=len(griz_unique)):
    g_id = int(g[4])
    if g_id == -99: continue    # Skip if not cosmos number
    
    # Check if already been fit
    f_ids = np.where(g_fits[:,0].astype(int) == g_id)[0]
    if len(f_ids) != 0:    # Already been fit
        griz_fits.append(g_fits[f_ids[0]])
        continue
    
    # Hasn't been fit
    else:
        a, w, l, r = fitDist(c20p["lp_zPDF"][g_id-1], c20p["lp_zPDF_l68"][g_id-1], c20p["lp_zPDF_u68"][g_id-1])
        #Check to see if fit converged
        f_med = skewnorm(a,scale=w,loc=l).median()
        if np.abs(f_med - c20p["lp_zPDF"][g_id-1] ) < 0.1:
            griz_fits.append([g_id, a,w,l,r])
        else:
            continue

griz_fits = np.array(griz_fits)
np.save("zFits/gFits.npy", griz_fits)

  0%|          | 0/10053 [00:00<?, ?it/s]

c:\Users\sikor\Anaconda3\envs\py38\lib\site-packages\scipy\stats\_distn_infrastructure.py:2176: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
c:\Users\sikor\Anaconda3\envs\py38\lib\site-packages\scipy\stats\_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
c:\Users\sikor\Anaconda3\envs\py38\lib\site-packages\scipy\stats\_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\Users\sikor\AppData\Local\Temp\ipykernel_22100\3998956261.py:5: RuntimeWarning: invalid value encountered in sqrt
  d = np.sqrt(  np.pi/2 * (1-var/wi**2))      # Delta parameter


In [55]:
### Pack the fits for use ###
g_params = []
b_params = []
for idx, c_id in enumerate(griz_unique[:,4]):
    idxs = np.where(griz_fits[:,0] == c_id)[0]
    if len(idxs) != 0:
        i = idxs[0]
        g_params.append([griz_fits[i][1],griz_fits[i][2], griz_fits[i][3]])
    else :
        b_params.append(idx)
    

g_params = np.array(g_params)
final_griz = np.delete(griz_unique, b_params, axis=0)

In [57]:
#### RUN THE MC ####
# ========================================================
# ========================================================
z_range = [2,3]         # Redshift range for 
plot_field = "./MC_iterations/griz_total/"
plot_zrange = "./MC_iterations/griz_Hyper/"

griz_z = final_griz[:,13].astype(float)         # orginal spec-z
griz_width = 46/14100*(1+griz_z)       # Width of the normal distribution to draw from

griz_med = griz_params[:,0]  # parameters for the p(z)
griz_l68 = griz_params[:,1]
griz_u68 = griz_params[:,2]


# Set the MC weights based on the quality flags
qfs = final_griz[:,-1]  
griz_weights = np.select( [qfs==5, qfs==4, qfs==3 ],
                [0.925, 0.818, 0.668],
                default=0)

# ========================================================
# ========================================================
# ========================================================
spec_mc["ID"][len(new_szs):len(new_szs) + len(final_griz)] = final_griz[:,4]     # update with cosmos IDs
which_z["ID"][len(new_szs):len(new_szs) + len(final_griz)] = final_griz[:,4]

for n in tqdm(range(niter)):

    gzs = np.random.normal(griz_z, griz_width)

    ## MC ##
    griz_ids, new_g, new_ids = MCz(1, gzs, griz_weights, z_range, my_PDF, 
                        verbose=False, alpha = griz_fits[:,1], omega = griz_fits[:,2], loc = griz_fits[:,3])
    
    new_gzs = new_g.flatten()
    # new_gzs[bad_griz] = -99
    
    spec_mc[f"MC_iter{n}"][len(new_szs):len(new_szs) + len(final_griz)] = new_gzs
    which_z[f"MC_iter{n}"][len(new_szs):len(new_szs) + len(final_griz)][new_ids] = 2*np.ones(len(new_ids))



# WRITE TO RESULT FILE ##

# # Update dtypes
# dtypes = [c20p.dtype.descr[0]] + [(f"MC_iter{n}", ">f8") for n in range(niter)]

# # Make array to fill
# write_arr = np.zeros(shape=(len(griz)), dtype=dtypes)

# write_arr["ID"] = griz[:,0]
# for n in range(niter):
#     write_arr[f"MC_iter{n}"] = new_gzs[:,n]

# np.save(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/grizli_MC_1000.npy", write_arr)

  0%|          | 0/100 [00:00<?, ?it/s]

---

## MC Common Objects

In [58]:
## Pack parameters of spectra from cosmos catalog ##
bad_com = [] # where parameters have a nan
com_fits = []
for idx, c_id in enumerate(sim_objs[:,0].astype(int)):

    if c_id == -99: continue    # Skip if not cosmos number
    
    # Check if already been fit
    f_ids = np.where(g_fits[:,0].astype(int) == c_id)[0]
    if len(f_ids) != 0:    # Already been fit
        com_fits.append(g_fits[f_ids[0]])
        continue
    
    # Hasn't been fit
    else:
        a, w, l, r = fitDist(c20p["lp_zPDF"][c_id-1], c20p["lp_zPDF_l68"][c_id-1], c20p["lp_zPDF_u68"][c_id-1])
        #Check to see if fit converged
        f_med = skewnorm(a,scale=w,loc=l).median()
        if np.abs(f_med - c20p["lp_zPDF"][c_id-1] ) < 0.1:
            com_fits.append([c_id, a,w,l,r])
        else:
            continue


com_fits = np.array(com_fits)
bad_com = np.array(bad_com)
np.save("zFits/cFits.npy", com_fits)

c:\Users\sikor\Anaconda3\envs\py38\lib\site-packages\scipy\stats\_distn_infrastructure.py:2176: RuntimeWarning: divide by zero encountered in divide
  x = np.asarray((x - loc)/scale, dtype=dtyp)
c:\Users\sikor\Anaconda3\envs\py38\lib\site-packages\scipy\stats\_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
c:\Users\sikor\Anaconda3\envs\py38\lib\site-packages\scipy\stats\_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc
C:\Users\sikor\AppData\Local\Temp\ipykernel_22100\3998956261.py:5: RuntimeWarning: invalid value encountered in sqrt
  d = np.sqrt(  np.pi/2 * (1-var/wi**2))      # Delta parameter


In [59]:
### Pack the fits for use ###
c_params = []
b_params = []
for idx, c_id in enumerate(sim_objs[:,0]):
    idxs = np.where(com_fits[:,0] == c_id)[0]
    if len(idxs) != 0:
        i = idxs[0]
        c_params.append([com_fits[i][1],com_fits[i][2], com_fits[i][3]])
    else :
        b_params.append(idx)
    

c_params = np.array(c_params)
final_c = np.delete(sim_objs, b_params, axis=0)

In [67]:
#### RUN THE MC ####
# ========================================================
# ========================================================
z_range = [2,3]         # Redshift range for 


## Weights

# Spectra weights
qfs = final_c[:,2] % 10      # 
spec_weights = np.select( [(qfs >=2.)&(qfs<3.),(qfs>=9.)&(qfs<10.), (qfs>=3.)&(qfs<5.) ],
                [0.7, 0.7, 0.993],
                default=0)

# Grizli weights 
qfg = final_c[:,-1]  
griz_weights = np.select( [qfg==5, qfg==4, qfg==3 ],
                [0.925, 0.818, 0.668],
                default=0)

# Combine
sim_weights = np.c_[spec_weights, griz_weights]

# Keep track of which flag is higher
max_id = np.argmax(sim_weights, axis=1)

# Sort the weights
sim_weights = np.sort(sim_weights, axis=1)


spec_mc["ID"][len(new_szs) + len(final_griz):len(new_szs) + len(final_griz) + len(final_c)] = final_c[:,0]     # update with cosmos IDs
which_z["ID"][len(new_szs) + len(final_griz):len(new_szs) + len(final_griz) + len(final_c)] = final_c[:,0]     # update with cosmos IDs

# ========================================================
# ========================================================
# ========================================================

for n in tqdm(range(niter)):

    # Draw random number for each object:
    mc_rns = np.random.random(size=len(sim_weights))

    # Choose specz, griz, or photoz
    z_choice = []   
    for rn_idx, rn in enumerate(mc_rns):
        sw = sim_weights[rn_idx]    # weights for this spectrum

        if rn < sw[1]: 
            z_choice.append(max_id[rn_idx]) # Choose better spectrum

        elif (rn >=sw[1]) and (rn < sw[1]+sw[0]*(1-sw[1])):
            z_choice.append(not(max_id[rn_idx]))    # Choose worse spectrum

        else:
            z_choice.append(2)  # Choose photoz

    
    # Make random grism redshifts
    g_rand = np.random.normal(final_c[:,3], 46/14100*(1+final_c[:,3]) )

    # Pick which redshift to use
    z_meds = np.select([z_choice == 0, z_choice == 1, z_choice == 2], 
                       [final_c[:,1],  g_rand, 2])
    
    # For "which_z" storage only
    wz = np.select([z_choice==0, z_choice==1, z_choice==2],
                   [1, 2, 0])
    
    # Assign weights
    ws = [0 if zi == 2 else 1 for zi in z_choice]


    ## MC ##
    _, new_sim , _ = MCz(1, z_meds, ws, z_range, my_PDF,
                       verbose=False, alpha = c_params[:,0], omega = c_params[:,1], loc = c_params[:,2])
    
    new_simz = new_sim.flatten()
    # new_simz[bad_com] = -99
    
    spec_mc[f"MC_iter{n}"][len(new_szs) + len(final_griz):len(new_szs) + len(final_griz) + len(final_c)] = new_simz
    which_z[f"MC_iter{n}"][len(new_szs) + len(final_griz):len(new_szs) + len(final_griz) + len(final_c)] = wz


  0%|          | 0/100 [00:00<?, ?it/s]

In [68]:
spec_mc = spec_mc[:len(new_szs) + len(final_griz) + len(final_c)]
which_z = which_z[:len(new_szs) + len(final_griz) + len(final_c)]

In [69]:
np.save(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/MC_spec.npy", spec_mc)
np.save(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/MC_which.npy", which_z)

----
----
----


# Fix up stuff

In [70]:
### Sort spectra by cosmos ID ###
sorted_spec = np.sort(spec_mc)
sorted_which_z = np.sort(which_z)

In [71]:
dtypes = [c20p.dtype.descr[0]] + [("zs", ">f8"), ("zg", ">f8")] + [(f"MC_iter{n}", ">f8") for n in range(niter)]

# Make array to fill
final_spec = np.zeros(shape=(len(sorted_spec)), dtype=dtypes)
final_which = np.zeros(shape=(len(sorted_spec)), dtype=dtypes)


final_spec["ID"] = sorted_spec["ID"]
final_which["ID"] = sorted_spec["ID"]



# Add original redshifts
zs = []
zg = []

for id in tqdm(final_spec["ID"]):
    # Get original specz
    spec_check = np.where(c20s[:,0] == id)[0]
    if len(spec_check)==1: zs.append(c20s[:,11][spec_check[0]])
    else: zs.append(-99)

    # get original griz-z
    griz_check = np.where(griz[:,4] == id)[0]
    if len(griz_check) ==1: zg.append(griz[:,13][griz_check[0]])
    else: zg.append(-99)

final_spec["zs"] = zs
final_which["zs"] = np.ones(len(zs))

final_spec["zg"] = zg
final_which["zg"] = 2*np.ones(len(zg))

for n in tqdm(range(niter)):
    final_spec[f"MC_iter{n}"] = sorted_spec[f"MC_iter{n}"]
    final_which[f"MC_iter{n}"] = sorted_which_z[f"MC_iter{n}"]

np.save(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/C20spec_MC_100.npy", final_spec)
np.save(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/C20spec_MC_100_choices.npy", final_which)

  0%|          | 0/48325 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [25]:
cosmos_file = fits.open(r"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/COSMOS2020_CLASSIC_R1_v2.0.fits")
c20p = cosmos_file[1].data

niter = 100

for run in range(1):

    mcs = np.load(rf"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/C20_MC_250_{run}.npy")


    dtypes = [c20p.dtype.descr[0]] + [("lp_zPDF", ">f8")] + [(f"MC_iter{n}", ">f8") for n in range(niter)]

    # Make array to fill
    final = np.zeros(shape=(len(mcs)), dtype=dtypes)

    final["ID"] = mcs["ID"]

    final["lp_zPDF"] = c20p["lp_zPDF"]

    for n in tqdm(range(niter)):
        final[f"MC_iter{n}"] = mcs[f"MC_iter{n}"]


    mcs = np.save(rf"C:/Users/sikor/OneDrive/Desktop/BigData/COSMOS2020/C20_MC_250_{run}_redo.npy", final)



  0%|          | 0/100 [00:00<?, ?it/s]